<a href="https://colab.research.google.com/github/AI-knows-your-rights/CS224-TC/blob/main/Ray_score_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Extracting the individual clauses and labels

In [2]:
# The DATA_DIR folder contains ~500 websites and their TC grade, their clauses and rating. In the documents subfolder there are raw Terms and Conditions converted to TXT.

DATA_DIR = "CS224-TC/data_all_202503130545112"

# The shared cases, points are in the data_downloader folder, double as a local cache.


In [3]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install 'accelerate>=0.26.0'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import os
import json

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from datasets import Dataset



In [5]:
# Data Loading Function
def load_clauses_data(data_dir):
    """
    Load clauses data from the specified directory
    Parameters:
        data_dir (str): Path to the directory containing service folders
    Returns:
        list: List of dictionaries containing clause data
    """
    all_clauses = []

    for service_folder in os.listdir(data_dir):
        service_path = os.path.join(data_dir, service_folder)

        if not os.path.isdir(service_path):
            continue

        clauses_file = os.path.join(service_path, 'clauses.json')
        if not os.path.exists(clauses_file):
            continue

        try:
            with open(clauses_file, 'r', encoding='utf-8') as f:
                data = json.load(f)

            if 'clauses' not in data or not data['clauses']:
                print(f"⚠️ WARNING: 'clauses' list is empty in '{service_folder}/clauses.json'")
                continue

            for clause in data['clauses']:
                if not all(key in clause for key in ['clause_text', 'description', 'rating']):
                    print(f"⚠️ WARNING: Skipping a clause in '{service_folder}' due to missing required fields")
                    continue

                clause_data = {
                    'service': service_folder,
                    'clause_text': clause['clause_text'],
                    'description': clause['description'],
                    'rating': clause['rating']
                }
                all_clauses.append(clause_data)

        except Exception as e:
            print(f"Error processing {service_folder}: {str(e)}")

    return all_clauses

In [6]:
rating_map = {
    'good': 0,
    'neutral': 1,
    'bad': 2,
    'blocker': 3,
}

In [7]:

if not os.path.exists(DATA_DIR):
    print(f"Directory '{DATA_DIR}' not found. Cloning repository...")
    # Remove existing directory if it exists
    !rm -rf CS224-TC
    # Clone the repository
    !git clone --depth 1 --filter=blob:none https://github.com/AI-knows-your-rights/CS224-TC.git
else:
    print(f"Directory '{DATA_DIR}' already exists. Skipping cloning.")

# !git clone --depth 1 --filter=blob:none --no-checkout https://github.com/AI-knows-your-rights/CS224-TC.git
# Enable sparse-checkout
#!git sparse-checkout init --cone
# Specify the folder you want to checkout (e.g., "your_folder")
#!git sparse-checkout set data_all_202503120623106


Directory 'CS224-TC/data_all_202503130545112' not found. Cloning repository...
Cloning into 'CS224-TC'...
remote: Enumerating objects: 3417, done.
remote: Counting objects: 100% (3417/3417), done.
remote: Compressing objects: 100% (2951/2951), done.
remote: Total 3417 (delta 709), reused 2748 (delta 405), pack-reused 0 (from 0)
Receiving objects: 100% (3417/3417), 918.08 KiB | 2.44 MiB/s, done.
Resolving deltas: 100% (709/709), done.
remote: Enumerating objects: 23889, done.
remote: Counting objects: 100% (23889/23889), done.
remote: Compressing objects: 100% (5767/5767), done.
remote: Total 23889 (delta 20035), reused 21884 (delta 18108), pack-reused 0 (from 0)
Receiving objects: 100% (23889/23889), 129.65 MiB | 20.31 MiB/s, done.
Resolving deltas: 100% (20035/20035), done.
Updating files: 100% (33210/33210), done.
Filtering content: 100% (3/3), 1.22 GiB | 58.36 MiB/s, done.


In [8]:
# Load and prepare the dataset
clauses_data = load_clauses_data(DATA_DIR)
df = pd.DataFrame(clauses_data)

# Convert ratings to numerical values
df['rating'] = df['rating'].map(rating_map)
print("\nRating distribution after conversion:")
print(df['rating'].value_counts().sort_index())

# Split into training and testing sets

training_size = round(df.shape[0] * 0.8)

train_df, test_df = train_test_split(df, train_size=training_size, random_state=42)

print(f"Training set size: {len(train_df)}")
print(f"Testing set size: {len(test_df)}")
print("\nRating distribution in training set:")
print(train_df['rating'].value_counts().sort_index())

⚠️ WARNING: 'clauses' list is empty in 'Kink.com/clauses.json'
⚠️ WARNING: 'clauses' list is empty in 'RESEARCHCHEMAIAL SWITZERLAND/clauses.json'

Rating distribution after conversion:
rating
0    3829
1    5926
2    4301
3     321
Name: count, dtype: int64
Training set size: 11502
Testing set size: 2875

Rating distribution in training set:
rating
0    3067
1    4739
2    3444
3     252
Name: count, dtype: int64


### BERT

In [9]:
model_name = "nlpaueb/legal-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=4,
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
print("GPU Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")


GPU Available: True
Using GPU: Tesla T4


In [11]:
# Move model to GPU if available
model = model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [12]:
# Define tokenization function

def tokenize_function(examples):
    # Convert the input to a list of strings and ensure it's properly formatted
    texts = [str(text) for text in examples["clause_text"]]  # Ensure text is string

    return tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors=None  # Important: keep this as None for batched processing
    )


In [13]:
def prepare_dataset(df):
    return Dataset.from_pandas(df)

train_raw_data = prepare_dataset(train_df)
test_raw_data = prepare_dataset(test_df)

In [14]:
print(train_raw_data)

print("Dataset features:", train_raw_data.features)
print("Sample row:", train_raw_data[0])


train_dataset = train_raw_data.map(tokenize_function, batched=True)
test_dataset = test_raw_data.map(tokenize_function, batched=True)

columns_to_remove = ['service', 'clause_text', 'description']
train_dataset = train_dataset.remove_columns(columns_to_remove)
test_dataset = test_dataset.remove_columns(columns_to_remove)
train_dataset = train_dataset.rename_column('rating', 'labels')
test_dataset = test_dataset.rename_column('rating', 'labels')

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])


Dataset({
    features: ['service', 'clause_text', 'description', 'rating', '__index_level_0__'],
    num_rows: 11502
})
Dataset features: {'service': Value(dtype='string', id=None), 'clause_text': Value(dtype='string', id=None), 'description': Value(dtype='string', id=None), 'rating': Value(dtype='int64', id=None), '__index_level_0__': Value(dtype='int64', id=None)}
Sample row: {'service': 'TubeBuddy', 'clause_text': 'We may use Pixels to automatically record certain technical information about your interactions when you visit the Site or otherwise engage with us, to help deliver Cookies on our Site, or count users who have visited the Site.', 'description': 'Tracking technologies (such as web beacons, tracking pixels...) are employed on users and/or the service assigns a unique ID to a browser or device to track its behaviour.', 'rating': 2, '__index_level_0__': 13546}


Map:   0%|          | 0/11502 [00:00<?, ? examples/s]

Map:   0%|          | 0/2875 [00:00<?, ? examples/s]

In [15]:
# Define metrics for evaluation
import torch.nn.functional as F

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Convert labels to PyTorch tensor before applying one_hot
    labels = torch.tensor(labels, dtype=torch.int64)
    labels = F.one_hot(labels, num_classes=4).float()  # Convert to one-hot and float

    # predictions = predictions.squeeze()
    predictions = torch.tensor(predictions).float()
    mse = ((predictions - labels) ** 2).mean()
    rmse = np.sqrt(mse.item()) # Convert mse to a python number for numpy

    # predicted_class = torch.argmax(predictions, dim=1)

    rmse = np.sqrt(mse)
    return {
        "mse": mse.item(), # Convert mse to a python number for json serialization
        "rmse": rmse
    }




In [16]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.05,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    no_cuda=False,  # Enable GPU
    fp16=True,
    metric_for_best_model="eval_loss",
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
# To track the training
!pip install weave
!wandb login


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.8/325.8 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 7.6 MB/s eta 0:00:00
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rayhu007 (rayhu007-stanford) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [18]:
# Initialize trainer
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Stop after 3 epochs of no improvement
    # run_name="ray-legal-bert-classification",
)

In [19]:
print(train_dataset)

Dataset({
    features: ['labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 11502
})


In [20]:
from transformers import get_scheduler

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_dataset) * 3,  # 3 epochs
)

In [ ]:
# Start training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: rayhu007 (rayhu007-stanford) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


In [ ]:
# Evaluate the model on test set
test_results = trainer.evaluate()
print("\nTest Results:")
print(test_results)

# Function to predict ratings for new clauses

def predict_rating(clause_text):
    # Check if clause_text is None and handle it gracefully
    if clause_text is None:
        print("Warning: Encountered a None clause_text. Returning -1.")  # Or any other default value
        return -1  # Or any other default value

    inputs = tokenizer(
        clause_text,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.logits.squeeze()

    predicted_class = torch.argmax(predictions).item()
    return predicted_class

# Test prediction with a sample clause
sample_clause = test_df['clause_text'].iloc[0]
predicted_rating = predict_rating(sample_clause)
actual_rating = test_df['rating'].iloc[0]

print("\nSample Prediction:")
print(f"Predicted Rating: {predicted_rating:.2f}")
print(f"Actual Rating: {actual_rating}")

In [ ]:

# Evaluate the model on the entire test dataset
predictions = trainer.predict(test_dataset)

# Access predicted labels and metrics
predicted_labels = np.argmax(predictions.predictions, axis=1)
metrics = predictions.metrics

print("\nTest Results (Full Dataset):")
print(metrics)

correct_predictions = np.sum(predicted_labels == test_df['rating'])
total_predictions = len(test_df)
accuracy = correct_predictions / total_predictions

print(f"\nOverall Accuracy: {accuracy:.2f}")



In [ ]:
# # Predict ratings for all samples in test_df and compare with actual ratings
# for index in range(len(test_df)):
#     sample_clause = test_df['clause_text'].iloc[index]  # Get the clause text
#     predicted_rating = predict_rating(sample_clause)   # Predict the rating
#     actual_rating = test_df['rating'].iloc[index]     # Get the actual rating

#     print(f"\nSample Prediction ({index + 1}): {sample_clause}")
#     print(f"Predicted Rating: {predicted_rating}")
#     print(f"Actual Rating: {actual_rating}")

In [ ]:
# Evaluate the model on the test dataset
results = trainer.evaluate()

# Print out the evaluation results
print(results)


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Get predictions and true labels
predictions = trainer.predict(test_dataset)
predicted_labels = predictions.predictions.argmax(axis=-1)  # For multi-class classification
true_labels = predictions.label_ids

# Compute confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Visualize confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["good", "neutral", "bad", "blocker"], yticklabels=["good", "neutral", "bad", "blocker"])
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm") #, disable=["parser", "ner"])
nlp.max_length = 1500000

def load_tc_document(tc_files):
    all_paragraphs = []

    for doc_path in tc_files:
        with open(doc_path, 'r', encoding='utf-8') as file:
            text = file.read()
        # A basic clause extraction logic, for example, assuming each clause ends with a period
        # clauses = sent_tokenize(text)
        paragraphs = text.split('\n\n')
        all_paragraphs.extend([paragraph.strip() for paragraph in paragraphs if paragraph.strip()])
    return all_paragraphs

In [ ]:
# Helper function to split a paragraph into smaller potential clauses (you can customize this logic)
def split_paragraph_into_clauses(paragraph):
    # Simple example: split by punctuation marks like period (".") or conjunctions
    # More sophisticated approaches can involve sentence segmentation or keyword matching
    return paragraph.split('. ')

In [ ]:
# !pip install transformers sentence-transformers sklearn
# from sentence_transformers import SentenceTransformer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load the pre-trained model (Sentence-BERT)
similarity_model = SentenceTransformer('all-MiniLM-L6-v2')
clause_texts = [clause['clause_text'] for clause in clauses_data if clause.get('clause_text') and clause['clause_text'].strip()]
clauses_embeddings = similarity_model.encode(clause_texts)

def sentences_to_clauses(paragraphs, threshold_paragraph=0.8, threshold_clause=0.9):
    results = []

    for paragraph in paragraphs:
        # Generate embedding for the current paragraph
        paragraph_embedding = similarity_model.encode([paragraph])

        # Compute cosine similarities between the paragraph and all known clauses
        similarities = cosine_similarity(paragraph_embedding, clauses_embeddings)

        # Find the highest similarity score
        max_similarity = np.max(similarities)

        # First, check the entire paragraph similarity (considering the whole paragraph)
        if max_similarity > threshold_paragraph:
            # If it's a valid match with a known clause, mark as a valid clause
            results.append({
                "sentence": paragraph,
                "is_clause": True,
                "similarity": max_similarity
            })
        else:
            # If not, break down the paragraph into smaller chunks (potential clauses)
            # For example, split the paragraph into sentences (or smaller chunks)
            potential_clauses = split_paragraph_into_clauses(paragraph)

            for chunk in potential_clauses:
                chunk_embedding = similarity_model.encode([chunk])
                similarities = cosine_similarity(chunk_embedding, clauses_embeddings)
                max_similarity_chunk = np.max(similarities)

                if max_similarity_chunk > threshold_clause:
                    # If chunk matches a clause, mark as valid clause
                    results.append({
                        "sentence": chunk,
                        "is_clause": True,
                        "similarity": max_similarity_chunk
                    })
                else:
                    # Otherwise, mark as not a clause
                    results.append({
                        "sentence": chunk,
                        "is_clause": False,
                        "similarity": max_similarity_chunk
                    })

    # Count valid clauses
    clause_count = len([result for result in results if result['is_clause']])

    print(f"Total paragraphs: {len(paragraphs)}")
    print(f"Total clauses: {clause_count}")

    return results




In [ ]:
# Predict clause rating using the pre-trained model
def predict_clause_rating(clause_text):
    # Tokenize the clause and convert it into input format for the model
    inputs = tokenizer(clause_text, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    # Convert the predicted class to a rating
    if predicted_class == 0:
        return "good"
    elif predicted_class == 1:
        return "neutral"
    elif predicted_class == 2:
        return "bad"
    elif predicted_class == 3:
        return "blocker"

In [ ]:
# Calculate the document grade based on clause scores
def calculate_website_grade(website, tc_files, actual_grade):

    sentences  = load_tc_document(tc_files)
    clauses = sentences_to_clauses(sentences)
    # Initialize counts for good, bad, and blocker clauses
    good_count = 0
    bad_count = 0
    blocker_count = 0

    # Loop through each clause and predict its rating
    for clause in clauses:
        # print('checking clause:')
        # print(clause)
        if clause.get('is_clause') and clause['is_clause']:
            rating = predict_clause_rating(clause['sentence'])
            if rating == "good":
                good_count += 1
            elif rating == "bad":
                bad_count += 1
            elif rating == "blocker":
                blocker_count += 1

    # Calculate the score: good - bad - 3 * blocker
    score = good_count - bad_count - 3 * blocker_count

    # Determine the grade based on the score
    if good_count + bad_count + blocker_count == 0:
        grade = 'N/A'
    elif score <= -10 or blocker_count > good_count:
        grade = 'E'
    elif blocker_count >= 3 or bad_count > good_count:
        grade = 'D'
    elif score < 5:
        grade = 'C'
    elif bad_count > 0:
        grade = 'B'
    else:
        grade = 'A'
    print(f"{website} graded as {grade}, actual grade is: {actual_grade}")
    return {
        "website": website,
        "score": score,
        "predicted_grade": grade,
        "actual_grade": actual_grade,
        "good_count": good_count,
        "bad_count": bad_count,
        "blocker_count": blocker_count
    }


In [ ]:
from os.path import isdir
# Function to process all documents in subfolders
def process_websites(folder_path):
    results = []

    # Loop through each website subfolder
    for subfolder in os.listdir(folder_path):
        if not os.path.isdir(os.path.join(folder_path, subfolder)):
            print(f"Skipping non-folder: {subfolder}")
            continue
        print(f"Processing folder: {subfolder}")
        details_path = os.path.join(folder_path, subfolder, 'details.json')
        if not os.path.exists(details_path):
            print(f"Details file not found for {subfolder}")
            continue

        try:
            with open(details_path, 'r') as f:
                details = json.load(f)
                if 'rating' in details:
                    actual_grade = details['rating']
        except json.JSONDecodeError:
            print(f"Error parsing JSON in {details_path}, skipping.")
            continue

        documents_folder = os.path.join(folder_path, subfolder, 'documents')

        # A dictionary for website subfolder
        tc_files = []
        # loop through all the TXT files in the documents folder
        for file in os.listdir(documents_folder):
            if not (file.endswith('.txt') and file.startswith('TC')):  # Process only TC_xx.txt files
                continue
            doc_path = os.path.join(documents_folder, file)
            tc_files.append(doc_path)
        # calculate the website grade based
        result = calculate_website_grade(subfolder, tc_files, actual_grade)
        results.append(result)

    return results

In [ ]:
results = process_websites(DATA_DIR)


In [ ]:

# Print the results
for result in results:
    print(f"Website: {result['website']}, Score: {result['score']}, Predicted Grade: {result['predicted_grade']}, Actual Grade: {result['actual_grade']}")


In [ ]:
# Initialize variables to track correct predictions
correct_predictions = 0
total_predictions = len(results)

# Iterate through the results and compare predicted and actual grades
for result in results:
    predicted_grade = result['predicted_grade']
    actual_grade = result['actual_grade']

    # Check if the predicted grade matches the actual grade
    if predicted_grade == actual_grade:
        correct_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / total_predictions * 100

# Print the accuracy
print(f"Accuracy: {accuracy:.2f}%")
